In [2]:
import pandas as pd 
import math
import numpy as np
from scipy import interpolate
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings

mpl.rc('font',family='Times New Roman') #везде TNR
warnings.filterwarnings("ignore")


In [ ]:
from pydantic import BaseModel
from typing import List

class DataPoint(BaseModel):
    """
    Координаты для построения графика 
    """
    x: float
    y: float

class BuildGdh(BaseModel):
    """
    Схема датасета с координатами для построения ГДХ
    """
    datasets: List[DataPoint]
    label: str

class BaseGDH:
    
    p_komer = 0.101325
    t_komer = 283

    def __init__(self, q_rate, p_in, kpd, freq, freq_nom_all, t_in, p_out, diam, r_value, freq_nom, mgth, stepen, p_title, k_value = 1.31, name=None): 
        self.q_rate = q_rate
        self.p_in = p_in
        self.diam = diam
        self.kpd = kpd 
        self.freq = freq
        self.t_in = t_in
        self.r_value = r_value
        self.k_value = k_value
        self.p_out = p_out
        self.freq_nom = freq_nom
        self.freq_nom_all = freq_nom_all
        self.mgth = mgth
        self.stepen = stepen
        self.p_title = p_title
        self.name = name
        self.f_kpd:np.poly1d=None  


    @classmethod
    def create_by_excel(cls, df):
        df = pd.read_excel(df, sheet_name=sheet_name)
        del_idx = df[df.iloc[:,0] == '//'].index[0]
        df_1 = df.iloc[:del_idx][~(df.iloc[:,0] == '/')]
        df_2 = df.iloc[del_idx+1:][~(df.iloc[:,0] == '/')]
        q_rate = df_1['V'].to_numpy()
        kpd = df_1['kpd'].to_numpy()
        freq = df_1['f'].to_numpy()  
        r_value = df_2[df_2.iloc[:,0] == 'R'].iloc[:,1].values
        t_in = df_2[df_2.iloc[:,0] == 'T'].iloc[:,1].values
        diam = df_2[df_2.iloc[:,0] == 'd'].iloc[:,1].values
        freq_nom = df_2[df_2.iloc[:,0] == 'fnom'].iloc[:,1].values
        freq_nom_all = freq / freq_nom
        if df_1['Pvh'][0] < df_2[df_2.iloc[:,0] == 'P'].iloc[:,1].values:
            p_in = df_1['Pvh'].to_numpy()
            p_out =  df_2[df_2.iloc[:,0] == 'P'].iloc[:,1].values
        else:
            p_out = df_1['Pvh'].to_numpy()
            p_in = df_2[df_2.iloc[:,0] == 'P'].iloc[:,1].values 
        mgth = df_2[df_2.iloc[:,0] == 'mgth'].iloc[:,1].values 
        stepen = df_2[df_2.iloc[:,0] == 'stepen'].iloc[:,1].values 
        p_title = df_2[df_2.iloc[:,0] == 'ptitle'].iloc[:,1].values 
        # print(kpd, freq, freq_nom_all)
        return cls(q_rate, p_in, kpd, freq, freq_nom_all, t_in[0], p_out, diam[0], r_value[0], freq_nom[0], mgth[0], stepen[0], p_title[0], k_value = 1.31, name=sheet_name)
    

    def __format__(self) -> str:
        return f'{self.mgth:.0f}-{self.p_title:.0f} {self.stepen} ({self.freq_nom:.0f} об/мин)'    

    @classmethod 
    def get_pltn(cls, p_in:float, t_in:float, r_value:float, z:float) -> float: 
        """Расчет плотности
        Args:
            p_in (float): Давление, МПА
            t_in (float): Температура, K
            r_value (float): Постоянная Больцмана поделеная на молярную массу
            z (float): Коэффициент сверсжимаемости
        Returns:
            float: плотность газа при указанные давлении и температуры, кг/м3
        """
        pltn = p_in * 10**6 / (z * r_value * t_in)
        return pltn
    

    @classmethod
    def get_z_val(cls, p_in:float, t_in:float, t_krit=190, p_krit=4.6) -> float: 
        """Расчет коэффициента сверсжимаемости
        Args:
            p_in (float): Давление, МПА
            t_in (float): Температура, К
            t_krit (int, optional): Критич. Температура, К {default = 190}
            p_krit (float, optional): Критич. Давление, МПа {default = 4.6}
        Returns:
            float: Значение сверхсжимаемости Z
        """
        z_val = 1 - 0.427 * p_in / p_krit * (t_in / t_krit)**(-3.688)        
        if type(z_val) == float:
            return 0.1 if z_val < 0 else z_val
        else:
            if type(z_val) == np.ndarray or type(z_val) == np.float64:
                return np.where(z_val < 0 , 0.1, z_val)
            else:
                return z_val
    

    @classmethod
    def get_volume_rate_from_press_temp(cls, q_rate:float, p_in:float, t_in:float, r_value:float) -> float:
        """Расчет объемного расхода
        Args:
            pltn_0 (float): Стандартная плотность, кг/м3
            q_rate (float): Комерческий расход, млн. м3/сут
            pltn_1 (float): Плотность, кг/м3
        Returns:
            float: Возвращяет обьемный расход, при указанных условиях, м3/мин
        """
        z_0 = cls.get_z_val(cls.p_komer, cls.t_komer, t_krit=190, p_krit=4.6)
        pltn_0 = cls.get_pltn(cls.p_komer, cls.t_komer, r_value, z_0)
        z_1 = cls.get_z_val(p_in, t_in, t_krit=190, p_krit=4.6)
        pltn_1 = cls.get_pltn(p_in, t_in, r_value, z_1)
        volume_rate = q_rate * 10**6 * pltn_0 / (24 * 60 * pltn_1)
        return volume_rate
   
   
    @classmethod 
    def get_u_val(cls, diam:float, freq:float) -> float:
        """Расчет угловой скорости
        Args:
            diam (float): Диаметр, м
            freq (float): Частота, об/мин
        Returns:
            float: Угловая скорость, м/с
        """
        u_val = freq * diam * math.pi / 60
        return u_val
    

    @classmethod
    def get_koef_rash_from_volume_rate(cls, diam:float, u_val:float, volume_rate:float) -> float:
        """Расчет коэффицента расхода
        Args:
            diam (float): Диаметр, м
            u_val (float): Угловая скорость, м/с
            volume_rate (float): Обьемный расход при заданных условиях, м3/мин
        Returns:
            float: Возвращяет коеффициент расхода при заданных условиях и текущей температуре, д.ед
        """
        koef_rash_ = 4 * volume_rate / (math.pi * diam**2 * u_val * 60) 
        return koef_rash_
    

    @classmethod
    def get_koef_nap(cls, dh:float, u_val:float) -> float:     
        """Расчет безразмерного коэффицента напора
        Args:
            dh (float): Изменение энтальпии, Дж
            u_val (float): Угловая скорость, м/мин
        Returns:
            float: Возврощяет коеффициент напора, при заданных условиях и текущей температуре, д.ед
        """
        koef_nap = dh  / u_val**2
        return koef_nap
    

    @classmethod   
    def get_dh(cls, p_in:float, r_value:float, t_in:float, comp:float, k_value:float, kpd:float) -> float: 
        """удельное изменение энтальпии
        Args:
            z_1 (float): Коэффициент сверсжимаемости
            r_value (float): Постоянная Больцмана поделеная на молярную массу
            t_in (float): Температура, K
            comp (float): Степень сжатия
            k_value (float): Коэф-т политропы, б.м.
            kpd (float): Политропный кпд, д.ед
        Returns:
            float: Необходимое для сжатия измение энтальпии, дж/кг
        """
        z_1 = cls.get_z_val(p_in, t_in, t_krit=190, p_krit=4.6)
        dh = z_1 * r_value * t_in * (comp**((k_value - 1) / (k_value * kpd)) -1) * (k_value * kpd) /(k_value - 1)
        return dh  


    @classmethod   
    def get_comp_r(cls, p_in:float, p_out:float) -> float: 
        """расчет степени сжатия
        Args:
            p_in (float): Давление на входе, МПа
            p_out (float): Давление на выходе, МПа
        Returns:
            float: Степень сжатия
        """
        comp_r = p_out / p_in
        return comp_r
        

    @classmethod
    def get_line_N_p_in(self, u_val:float, z_in:float, r_val:float, t_in:float, koef_rash:float, koef_nap:float, kpd:float, diam:float) -> float:
        return (u_val ** 3) * koef_nap * koef_rash * math.pi * (diam**2) / (4 * z_in * r_val * t_in * kpd) * 10 ** 3        
    

    @classmethod
    def get_volume_rate_from_koef_rash(self, k_rash, diam, freq): 
        return k_rash * math.pi * diam**2 * (diam * math.pi)/4 * freq #обьемный расход из коэффициента расхода
    

    @classmethod
    def get_comp_ratio_from_koef_nap(self, koef_nap, freq, diam, kpd, t_in:float, z_in:float, r_val:float, k_value): # получение P2/P1 
        u_val = freq * diam * math.pi / 60
        dh =  koef_nap * u_val**2
        m_t =  (k_value - 1) / (k_value * kpd)
        return (dh * m_t / (z_in * r_val * t_in) + 1) ** (1 / m_t)

       
    def get_koef_rash(self, koef_rash:np.array):
        k_rash_min = np.min(koef_rash)
        k_rash_max = np.max(koef_rash)

        res_min = minimize_scalar(lambda x: -self.f_kpd(x), bounds=[k_rash_min, k_rash_max])
        kpd_max = -res_min.fun 
        k_rash_kpd_max = res_min.x 

        # kpd_range_left = np.linspace(self.f_kpd(k_rash_min), kpd_max, 3)
        # kpd_range_right = np.linspace(kpd_max-0.01, self.f_kpd(k_rash_max), 3)
        
        kpd_range_left = np.arange(self.f_kpd(k_rash_min), kpd_max+0.025, 0.005)

        if kpd_range_left.size > 3:
            kpd_range_left = np.linspace(self.f_kpd(k_rash_min), kpd_max, 4)

        kpd_range_right = np.arange(kpd_max-0.01, self.f_kpd(k_rash_max), -0.01)

        if kpd_range_right.size > 3:
            kpd_range_right = np.linspace(kpd_max-0.01, self.f_kpd(k_rash_max), 4)

        kpd_range = np.array([
            self.f_kpd(k_rash_min),
            *np.round(kpd_range_left,10)[1:-1],
            kpd_max,
            *np.round(kpd_range_right,10)[1:-1],
            self.f_kpd(k_rash_max)])

        f_left = interpolate.interp1d(
            x=self.f_kpd(np.linspace(k_rash_min, k_rash_kpd_max)),
            y=np.linspace(k_rash_min, k_rash_kpd_max)
            )

        f_right = interpolate.interp1d(
            x=self.f_kpd(np.linspace(k_rash_kpd_max, k_rash_max)),
            y=np.linspace(k_rash_kpd_max, k_rash_max)
            )

        k_koef_rash_range = np.array([
            *f_left(kpd_range[0:kpd_range_left.size]),
            *f_right(kpd_range[kpd_range_left.size:]),
            ])
        return k_koef_rash_range
    

    def get_summry(self):
        volume_rate = self.get_volume_rate_from_press_temp(self.q_rate, self.p_in, self.t_in, self.r_value)
        comp = self.get_comp_r(self.p_in, self.p_out)

        u_val = self.get_u_val(self.diam, self.freq)
        koef_rash = self.get_koef_rash_from_volume_rate(self.diam, u_val, volume_rate).astype(np.float64)

        dh = self.get_dh(self.p_in, self.r_value, self.t_in, comp, self.k_value, self.kpd)
        koef_nap = self.get_koef_nap(dh, u_val).astype(np.float64)

        c00_kpd = np.polyfit(x=koef_rash, y=self.kpd.astype(np.float64), deg=4)        
        self.f_kpd = np.poly1d(c00_kpd)
        c00_nap = np.polyfit(x=koef_rash, y=koef_nap, deg=4)
        self.f_nap = np.poly1d(c00_nap) 

        koef_rash_ = self.get_koef_rash(koef_rash)
        kpd_ = self.f_kpd(koef_rash_)
        koef_nap_ = self.f_nap(koef_rash_)

        freq_base_nom = [1.05, 1.0, 0.95, 0.90, 0.85, 0.80, 0.75, 0.7]
        freq_base = [x * self.freq_nom for x in freq_base_nom]
        volume_rate_x = self.get_volume_rate_from_koef_rash(koef_rash_, self.diam, np.vstack(freq_base))

        z_in = self.get_z_val(self.p_in, self.t_in)  
        comp_y = self.get_comp_ratio_from_koef_nap(koef_nap_, np.vstack(freq_base), self.diam, kpd_, self.t_in, z_in[0], self.r_value, self.k_value)
        # power_line = self.get_line_N_p_in(u_val, z_in, self.r_value, self.t_in, koef_rash_, koef_nap_, kpd_, self.diam)

        res = {
            # 'N/pн': power_line,
            'freq_nom_all': freq_base_nom,
            'comp': [comp_ for comp_ in comp_y],
            'volume_rate': [volume_rate_x for volume_rate_x in volume_rate_x],
            'title': self.__format__()
            }
        res = pd.DataFrame(res)

        grouped_dfs = [group.reset_index(drop=True) for _, group in res.groupby('freq_nom_all')]
        lst_df_2 = []
        lst_buil_gdh_freq = []
        for df in grouped_dfs:
            df_freq = df[['volume_rate', 'comp']].T
            df_freq = pd.DataFrame({**{str(i): df_freq[0].str[i] 
                                for i in range(len(df_freq.loc['comp'][0]))}})
            df_kpd_1 = df_freq.T.stack()
            lst_df_2.append(df_kpd_1)
            datasets_freq = []
            for volume_rate, comp in zip(df_freq.loc['volume_rate'], df_freq.loc['comp']):
                dataset_freq = DataPoint(
                            x=volume_rate, 
                            y=comp
                            ) 
                datasets_freq.append(dataset_freq)

            buil_gdh_freq = BuildGdh(
                label=f"{df['freq_nom_all'][0]}",
                datasets=datasets_freq
                    )
            lst_buil_gdh_freq.append(buil_gdh_freq)

            lst_buil_gdh_kpd = []
            df_kpd_2 = pd.concat(lst_df_2, axis=1) 
            for i, label in zip(range(len(df_kpd_2) // 2), kpd_):
                volume_rate_row = df_kpd_2.iloc[2*i]
                comp_row = df_kpd_2.iloc[2*i+1]

                datasets_kpd = []
                for col in df_kpd_2.columns:
                    volume_rate = volume_rate_row[col]
                    comp = comp_row[col]
                    dataset_kpd = DataPoint(
                            x=volume_rate, 
                            y=comp)            
                    datasets_kpd.append(dataset_kpd)
                
                buil_gdh_kpd = BuildGdh(
                    label=f"{label}",
                    datasets=datasets_kpd
                    )
                lst_buil_gdh_kpd.append(buil_gdh_kpd)

        return [df_freq,              
                df_kpd_2]
 
 
    @classmethod
    def get_plot_gdh(self, data1, data2, data3, name):
        # print(data1, data2, data3)
        fig, ax = plt.subplots()

        for f_nom in data1['n/nном'].unique():
            sup_data = data1[data1['n/nном'] == f_nom]
            x = sup_data.iloc[0, 1:].astype(float).values
            y = sup_data.iloc[1, 1:].astype(float).values
            degree = 4  # Степень полинома
            coefficients = np.polyfit(x, y, degree)
            polynomial = np.poly1d(coefficients)   
            x_smooth = np.linspace(x.min(), x.max(), 300)
            y_smooth = polynomial(x_smooth)       
            ax.plot(x_smooth, y_smooth, color = 'black', linestyle='-')      
            if f_nom == 1: 
                ax.plot(x_smooth, y_smooth, linestyle='-', color = 'deepskyblue')
            if f_nom == 1.05: 
                ax.plot(x_smooth, y_smooth, linestyle='-', color = 'red')  
            ax.text(sup_data.iloc[0, 1], sup_data.iloc[1, 1], round(f_nom, 2), ha = 'right', va = 'baseline', fontsize=9)
        
        for i in range(0, len(data2), 2):
            sup_data = data2.iloc[i:i+2]
            x = sup_data.iloc[0, 1:].astype(float).values
            y = sup_data.iloc[1, 1:].astype(float).values
            degree = 4  # Степень полинома
            coefficients = np.polyfit(x, y, degree)
            polynomial = np.poly1d(coefficients)   
            x_smooth = np.linspace(x.min(), x.max(), 300)
            y_smooth = polynomial(x_smooth)         
            ax.plot(x_smooth, y_smooth, color = 'black', linestyle='-')
            end_value = len(sup_data.iloc[0, 1:])
            ax.text(sup_data.iloc[0, end_value], sup_data.iloc[1, end_value], round(data2['kpd'][i], 2), ha = 'left', va = 'bottom', fontsize=9)
            
        if data3.empty:
            pass
        else: 
            for power_l in data3['N/pн'].unique():
                sup_data = data3[data3['N/pн'] == power_l]
                ax.plot(sup_data.iloc[0, 1:], sup_data.iloc[0, 1:], color = 'black', linestyle='--', linewidth=1)
                end_value = len(sup_data.iloc[0, 1:])
                ax.text(sup_data.iloc[0, end_value], sup_data.iloc[1, end_value], round(power_l, 2), ha = 'left', va = 'top', fontsize=9)
        
        plt.title(f"{name}", fontsize=10)
        ax.set_xlabel('Q, м\u00b3/мин', fontsize=10, loc='right')
        ax.set_ylabel(r'$\epsilon$', fontsize=10, rotation=0, loc='top')
        ax.set_axisbelow(True)
        ax.grid(color='lightgray', linestyle='dashed')
        return plt

    
df = pd.ExcelFile('../media/Оцифрованные СПЧ.xlsx')
for sheet_name in df.sheet_names:
    # print(sheet_name)
    dimKoef = BaseGDH.create_by_excel(df)
    res = dimKoef.get_summry()
    print(res)
    # dimKoef.get_plot_gdh(*res, sheet_name)

0    146.008764
Name: (0, volume_rate), dtype: float64 0    1.284994
Name: (0, comp), dtype: float64
0    183.732628
Name: (1, volume_rate), dtype: float64 0    1.280865
Name: (1, comp), dtype: float64
0    210.870196
Name: (2, volume_rate), dtype: float64 0    1.271698
Name: (2, comp), dtype: float64
0    264.007778
Name: (3, volume_rate), dtype: float64 0    1.239448
Name: (3, comp), dtype: float64
0    319.488576
Name: (4, volume_rate), dtype: float64 0    1.189134
Name: (4, comp), dtype: float64
0    339.725117
Name: (5, volume_rate), dtype: float64 0    1.167733
Name: (5, comp), dtype: float64
0    375.016883
Name: (6, volume_rate), dtype: float64 0    1.12796
Name: (6, comp), dtype: float64
0    146.008764
1    156.437962
Name: (0, volume_rate), dtype: float64 0    1.284994
1    1.331479
Name: (0, comp), dtype: float64
0    183.732628
1    196.856388
Name: (1, volume_rate), dtype: float64 0    1.280865
1    1.326685
Name: (1, comp), dtype: float64
0    210.870196
1    225.932353
